In [1]:
import sys

sys.path.append("..")

In [ ]:
import time
import rasterio
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt

from textwrap import fill
from functools import partial
from shapely.geometry import LineString, Point, box

from tests.raster_processing.clip_test import raster_path as SMALL_RASTER_PATH

from eis_toolkit.vector_processing.distance_computation import distance_computation, distance_computation_optimized

In [3]:
with rasterio.open(SMALL_RASTER_PATH) as raster:
    SMALL_RASTER_PROFILE = raster.profile

geodataframe_with_raster_crs = partial(gpd.GeoDataFrame, crs=SMALL_RASTER_PROFILE["crs"])

EXPECTED_SMALL_RASTER_SHAPE = SMALL_RASTER_PROFILE["height"], SMALL_RASTER_PROFILE["width"]

POINT_GEOMETRIES_WITHIN_SMALL_RASTER = geodataframe_with_raster_crs(
    geometry=[
        Point(384745.000, 6671375.000),
        Point(384800.000, 6671375.000),
    ]
)
LINE_GEOMETRIES_WITHIN_SMALL_RASTER = geodataframe_with_raster_crs(
    geometry=[
        LineString([Point(384745.000, 6671375.000), Point(384800.000, 6671375.000)]),
        LineString([Point(384745.000, 6671375.000), Point(384745.000, 6671375.000)]),
    ]
)
POLYGON_GEOMETRIES_WITHIN_SMALL_RASTER = geodataframe_with_raster_crs(
    geometry=[
        box(384744.000, 6671272.000, 384764.000, 6671292.000),
        box(384784.000, 6671280.000, 384800.000, 6671300.000),
    ]
)


test_cases = [
    (
        "point_geometries_within_small_raster",
        SMALL_RASTER_PROFILE,
        POINT_GEOMETRIES_WITHIN_SMALL_RASTER,
        EXPECTED_SMALL_RASTER_SHAPE,
        0.0,
        107.83784122468327,
    ),
    (
        "line_geometries_within_small_raster",
        SMALL_RASTER_PROFILE,
        LINE_GEOMETRIES_WITHIN_SMALL_RASTER,
        EXPECTED_SMALL_RASTER_SHAPE,
        0.0,
        107.83784122468327,
    ),
    (
        "polygon_geometries_within_small_raster",
        SMALL_RASTER_PROFILE,
        POLYGON_GEOMETRIES_WITHIN_SMALL_RASTER,
        EXPECTED_SMALL_RASTER_SHAPE,
        0.0,
        91.0,
    ),
]


In [ ]:
# TEST OLD DISTANCE COMPUTATION IMPLEMENTATION
for test, raster_profile, geodataframe, _, _, _ in test_cases:
    print(f"Running test: {test}")
    start = time.time()
    result = distance_computation(raster_profile=raster_profile, geodataframe=geodataframe)
    end = time.time()
    print(f"Time took to perform distance computation: {end-start} s.")

Running test: point_geometries_within_small_raster
Time took to perform distance computation: 0.04545879364013672 s.
Running test: line_geometries_within_small_raster
Time took to perform distance computation: 0.029377222061157227 s.
Running test: polygon_geometries_within_small_raster
Time took to perform distance computation: 0.02394890785217285 s.


In [14]:
# TEST NEW DISTANCE COMPUTATION IMPLEMENTATION USING SPATIAL INDEXING AND NUMBA
for test, raster_profile, geodataframe, _, _, _ in test_cases:
    print(f"Running test: {test}")
    start = time.time()
    result = distance_computation_optimized(raster_profile=raster_profile, geodataframe=geodataframe)
    end = time.time()
    print(f"Time took to perform distance computation: {end-start} s.")

Running test: point_geometries_within_small_raster
Time took to perform distance computation: 0.003940582275390625 s.
Running test: line_geometries_within_small_raster
Time took to perform distance computation: 0.0025339126586914062 s.
Running test: polygon_geometries_within_small_raster
Time took to perform distance computation: 0.0016758441925048828 s.


In [ ]:
def _plot_image(ax, data, title, transform):
    plot.show(data, transform=transform, ax=ax)
    plt.subplots_adjust(hspace=0.4)
    ax.set_title(fill(title, width=25))
    norm = plt.Normalize(vmax=np.nanmax(data), vmin=np.nanmin(data))
    cmap = matplotlib.cm.viridis
    plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes = axes.flatten()

